In [2]:
import pandas as pd
import seaborn as sns
import re
import matplotlib

C:\Users\sante\AppData\Local\Temp\ipykernel_9736\1466026045.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,name,price,params,desc,region
2744141438,"EXEED TXL 2.0 AMT, 2023",4680000,"2.0 AMT (197 л.с.), внедорожник, полный, бензин",Покупайте Exeed с выгодой до 1 150 000 ₽ в лиз...,Санкт-Петербург
3464828016,"EXEED RX 2.0 AMT, 2023",5450000,"2 AMT (249 л.с.), внедорожник, полный, бензин",Сильвер официальный дилер Exeed предлагает уни...,Челябинск
3493473950,"EXEED TXL 2.0 AMT, 2023",4680000,"2.0 AMT (197 л.с.), внедорожник, полный, бензин",Выгода На Exeed Txl ДО 1 590 000 Рублей! Офици...,Краснодар
3467133132,"EXEED TXL 1.6 AMT, 2023",4000000,"1.6 AMT (186 л.с.), внедорожник, полный, бензин",Влюбитесь в каждое мгновенье с новым автомобил...,Санкт-Петербург
3278163543,"EXEED TXL 1.6 AMT, 2023",4260000,"1.6 AMT (186 л.с.), внедорожник, полный, бензин","Exeed — бренд премиальных автомобилей, созданн...",Ростов-на-Дону


In [3]:
def prepare_data(path):

    df = pd.read_csv(path, index_col=0)
    df[['mileage', 'engine_capacity', 'transmission', 'engine_power', 'body_type', 'wheel', 'fuel_type', 'model', 'model_2', 'year']] = None

    for i, row in df.loc[:, ['name', 'params', 'region']].iterrows():

        try:
            #Предобработка params
            mileage = re.search(r'([\d ]+) км,', row['params'], re.IGNORECASE)
            if mileage == None:
                mileage = 0
            else:
                df.loc[i, 'mileage'] = mileage.group(1).replace(' ', '')

            engine_capacity = re.search(r'([\d\.]+) [A-Z]+', row['params'])

            if engine_capacity != None:
                df.loc[i, 'engine_capacity'] = engine_capacity.group(1)

            df.loc[i, 'transmission'] = re.search(r'MT|CVT|AMT|AT', row['params'])[0]
            df.loc[i, 'engine_power'] = re.search(r'\((\d+) л\.с\.\),', row['params']).group(1)


            df.loc[i, 'body_type'] = re.search(r'([\w -]+), (полный|задний|передний)', row['params'], re.IGNORECASE).group(1)
            df.loc[i, 'wheel'] = re.search(r'полный|задний|передний', row['params'], re.IGNORECASE)[0]
            df.loc[i, 'fuel_type'] = re.search(r'бензин|дизель|электро|гибрид|газ', row['params'], re.IGNORECASE)[0]


            #предобработка name
            df.loc[i, 'model'] = re.search(r'[\w-]+ ', row['name'], re.IGNORECASE)[0]
            df.loc[i, 'model_2'] = re.search(r'[\w-]+ ([^,]*?) [\d\. ]*(AT|MT|AMT|CVT),', row['name'], re.IGNORECASE).group(1)        
            df.loc[i, 'year'] = re.search(r'\d{4}', row['name'])[0]

            #предобработка region
            df.loc[i, 'region'] = re.search(r'([^,]+),?', row['region'], re.IGNORECASE).group(1)

        except Exception as ex:
            print(row)
            print(ex)

    return df

In [5]:
df = pd.DataFrame()
for i in range(28):
    if i == 0:
        res = prepare_data(f'./datasets/data.csv')
    else:
        res = prepare_data(f'./datasets/data{i}.csv')
    
    df = pd.concat([df, res])


name                                          Audi A6, 2011
params    186 000 км, 2.0 CVT (180 л.с.), седан, передни...
region                              Иркутская обл., Иркутск
Name: 2978798022, dtype: object
'NoneType' object has no attribute 'group'
name                                           BMW X5, 2008
params    300 000 км, 3.0 AT (272 л.с.), внедорожник, по...
region                                                Минск
Name: 2634543434, dtype: object
'NoneType' object has no attribute 'group'
name         Changan Lamore 1.5 AMT, 2023
params    1.5 AMT седан, передний, бензин
region               поселение Московский
Name: 3746407298, dtype: object
'NoneType' object has no attribute 'group'
name                                Chevrolet Lacetti, 2010
params    280 000 км, 1.6 MT (109 л.с.), хетчбэк, передн...
region    Курская обл., Мантуровский р-н, 2-й Засеймский...
Name: 2476088237, dtype: object
'NoneType' object has no attribute 'group'
name                              

In [11]:
df[df.isnull() == True]


,name,price,params,desc,region,mileage,engine_capacity,transmission,engine_power,body_type,wheel,fuel_type,model,model_2,year
3883899162,"Audi A4 1.4 AMT, 2017, 24 831 км",2570000,"24 831 км, 1.4 AMT (150 л.с.), седан, передний...","Данный автомобиль прошел юридическую проверку,...",Москва,24831,1.4,AMT,150,седан,передний,бензин,Audi,A4,2017
3719132447,"Audi A7 2.0 AMT, 2016, 116 401 км",2595000,"116 401 км, 2.0 AMT (249 л.с.), лифтбек, полны...",Дополнительная скидка до 300 000 рублей при по...,Москва,116401,2.0,AMT,249,лифтбек,полный,бензин,Audi,A7,2016
3662013276,"Audi A3 1.4 AMT, 2013, 98 000 км",1299000,"98 000 км, 1.4 AMT (122 л.с.), седан, передний...",Оставить заявку на рассрочку Оставить заявку н...,Москва,98000,1.4,AMT,122,седан,передний,бензин,Audi,A3,2013
3612342601,"Audi Q7 3.0 AT, 2021, 78 000 км",6999000,"78 000 км, 3.0 AT (231 л.с.), внедорожник, пол...","Автомобиль из Германии. Идеальное состояние, п...",Москва,78000,3.0,AT,231,внедорожник,полный,дизель,Audi,Q7,2021
3566107102,"Audi A6 2.0 AMT, 2017, 172 000 км",2279000,"172 000 км, 2 AMT (249 л.с.), седан, полный, б...",Оставить заявку на рассрочку Оставить заявку н...,Москва,172000,2,AMT,249,седан,полный,бензин,Audi,A6,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3527900766,"Audi A7 3.0 AMT, 2015, 123 629 км",2299000,"123 629 км, 3.0 AMT (245 л.с.), лифтбек, полны...",Все заявленные автомобили находятся в наличии....,Москва,123629,3.0,AMT,245,лифтбек,полный,дизель,Audi,A7,2015
3100422198,"Audi Q7 3.0 AT, 2022",11934500,"3.0 AT (249 л.с.), внедорожник, полный, дизель",Новый Audi Q7. 45 Tdi Quattro (183 квт/249 л. ...,Нижний Новгород,None,3.0,AT,249,внедорожник,полный,дизель,Audi,Q7,2022
3542692559,"Audi A4 2.0 AMT, 2015, 87 845 км",1979700,"87 845 км, 2.0 AMT (225 л.с.), седан, полный, ...",Полный комплект документов. Птс оригинал. Подт...,Котельники,87845,2.0,AMT,225,седан,полный,бензин,Audi,A4,2015
3607596883,"Audi A6 Allroad Quattro 3.0 AT, 2008, 147 000 км",1249000,"147 000 км, 3.0 AT (233 л.с.), универсал, полн...","Все автомобили, представленные в продаже, прох...",Москва,147000,3.0,AT,233,универсал,полный,дизель,Audi,A6 Allroad Quattro,2008
